# Hamiltonian MC
Useful for multidimensional approximations, like parameter estimations

Based on: https://towardsdatascience.com/python-hamiltonian-monte-carlo-from-scratch-955dba96a42d

In [3]:
import numpy as np
import random
import scipy.stats as st
import matplotlib.pyplot as plt

Fontconfig warning: ignoring UTF-8: not a valid region tag


In [4]:
def normal(x, mu, sigma):
    numerator = np.exp(-1* ((x-mu)**2)/2*sigma**2)
    denom = sigma * np.sqrt(2*np.pi)
    return numerator/denom

In [7]:
# test 
normal(np.array([5,3]), np.array([2,3]), np.array([3,4]))

array([3.42659119e-19, 9.97355701e-02])

In [ ]:
def neg_log_prob(x, mu, sigma):
    return -1 * np.log(normal(x=x, mu=mu, sigma=sigma))

In [8]:
def HMC(
    mu=0.0,
    sigma=1.0,
    path_len=1,
    step_sz=0.25,
    init_pos=0.0,
    epochs=1_000
):
    # setup
    steps = int(path_len/step_sz) # tricky to tune
    samples = [init_pos]
    momentum_dist = st.norm(0, 1)

    # generate samples
    for e in range(epochs):
        q0 = np.copy(samples[-1])
        q1 = np.copy(q0)
        p0 = momentum_dist.rvs() # sample distro
        p1 = np.copy(p0)
        dVdQ = -1*(q0-mu)/(sigma**2) # gradient of PDF wrt position (q0) aka potential energy wrt position

        # leapfrog integration begin
        for s in range(steps):
            p1 += step_sz*dVdQ/2 # pot energy up, kin energy down, first half step
            q1 += step_sz*p1 # pos inc as function of momentum
            p1 += step_sz*dVdQ/2 # second half step
        # leapfrog end
        p1 = -1*p1; # flip momentum for reversibility

        # metropolis acceptance
        q0_nlp = neg_log_prob(x=q0, mu=mu, sigma=sigma)
        q1_nlp = neg_log_prob(x=q1, mu=mu, sigma=sigma)

        p0_nlp = neg_log_prob(x=p0, mu=0, sigma=1)
        p1_nlp = neg_log_prob(x=p1, mu=0, sigma=1)

        # Account for negatives and log(prob)
        target = q0_nlp - q1_nlp
        adjustment = p1_nlp - p0_nlp
        acceptance = target + adjustment
        event = np.log(random.uniform(0, 1))
        if event <= acceptance:
            samples.append(q1)
        else:
            samples.append(q0)
    return samples


In [14]:
st.norm(0, 1).rvs()

-0.280704643005934